This notebook is currently running on a local machine. Some functionality, like file uploading using `google.colab.files.upload()`, is only available when running in a Google Colab environment.

## System Requirements

* Linux machine (Ubuntu 20.04 or later)
* GPU card (A6000 45 GB)
* Minimum CUDA Toolkit version: 11.8

### **Extract Frames from the video using ffmpeg**


In [ ]:
VIDEO_PATH="/data/ankit_phd/Tutorial/iisc_scene/iisc.mp4"
OUTPUT_DIR="/data/ankit_phd/Tutorial/iisc_scene"
%cd {OUTPUT_DIR}
%mkdir {OUTPUT_DIR}/images
#Adjust the FPS such that you have roughly 120 frames. This will fit in 48 GB card. If you have more memory, increase the fps accordingly
!ffmpeg -i {VIDEO_PATH} -vf fps=0.60 images/%05d.png

## Clone the VGGT codebase

### Subtask:
Clone the VGGT codebase from its repository.



```
!git clone https://github.com/facebookresearch/vggt.git
```




## Install dependencies

Install the required libraries and dependencies for running `demo_colmap`.



```
!pip install -r requirements.txt
!pip install -r requirements_demo.txt
```



In [ ]:
!which pip

## **Run demo_colmap to estimate camera poses and sparse point cloud**

Run the `demo_colmap` script from the VGGT codebase on the extracted frames.

In [ ]:
SCENE_DIR="/data/ankit_phd/Tutorial/iisc_scene"
VGGT_DIR="/data/ankit_phd/Projects/VGGT_Exploration/vggt"

In [ ]:
#Run the VGGT for getting the COLMAP
%cd {VGGT_DIR}
!CUDA_VISIBLE_DEVICES=1 python demo_colmap.py --scene_dir={SCENE_DIR} --use_ba
print(f"Current working directory: %pwd")

In [ ]:
#Small hack to make output of VGGT compatible with 3dgs
%mkdir {SCENE_DIR}/sparse/0
%mv {SCENE_DIR}/sparse/*.bin {SCENE_DIR}/sparse/0
%mv {SCENE_DIR}/sparse/*.ply {SCENE_DIR}/sparse/0

## Run 3D Gaussian Splatting (3DGS)

### Subtask:
Run 3D Gaussian Splatting (3DGS) using the estimated camera poses and sparse point cloud output generated in the previous step.

In [ ]:
NVS_DIR="/data/ankit_phd/Projects/gsplat/examples"
%cd {NVS_DIR}
OUTPUT_DIR="/data/ankit_phd/data/Samsung_Tutorial/output_gsplat"
!CUDA_VISIBLE_DEVICES=1 python simple_trainer.py default --data_dir {SCENE_DIR} --data_factor 1 --result_dir {OUTPUT_DIR}

## Visualize 3D Gaussian Splatting (3DGS)

Use Viser to visualize the final output.

In [ ]:
NVS_DIR="/data/ankit_phd/Projects/gsplat/examples"
%cd {NVS_DIR}
CKPT_PATH="/data/ankit_phd/Projects/gsplat/examples/{{OUTPUT_DIR}/ckpts/ckpt_29999_rank0.pt"
!CUDA_VISIBLE_DEVICES=1  python simple_viewer.py --ckpt  {CKPT_PATH} --port 8081

/home/test/anaconda3/envs/3dgs/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/data/ankit_phd/Projects/gsplat/examples
Number of Gaussians: 2308746
╭──────────────── viser ────────────────╮
│             ╷                         │
│   HTTP      │ http://localhost:8081   │
│   Websocket │ ws://localhost:8081     │
│             ╵                         │
╰───────────────────────────────────────╯
Viewer running... Ctrl+C to exit.
^C
Traceback (most recent call last):
  File "/data/ankit_phd/Projects/gsplat/examples/simple_viewer.py", line 248, in <module>
    cli(main, args, verbose=True)
  File "/home/test/anaconda3/envs/3dgs/lib/python3.10/site-packages/gsplat/distributed.py", line 360, in cli
    return _distributed_worker(0, 1, fn=fn, args=args)
  File "/home/test/anaconda3/envs/3dgs/lib/python3.10/site-packages/gsplat/distributed.py", line 295, in _distributed_worker
    fn(local_rank, world_rank, world_size, args)
  File "/data/ankit_phd/Projects/gsplat/examples/simple_viewer.py", line 213, in main
    time.sleep(100000)
KeyboardInterrupt


KeyboardInterrupt: 